# Linearization

In [147]:
from modelon.impact.client import Client
import ipywidgets as widgets
from IPython.display import display

In [144]:
global experiment_definition
client = Client()
workspaces = client.get_workspaces()
workspace_map = {workspace.definition.name:workspace.id for workspace in workspaces}

In [145]:
workspace_sel=widgets.Dropdown(
    options=workspace_map.keys(),
    value="TestCF",
    description='Workspaces',
    disabled=False,
)
output = widgets.Output()
print('Select a workspace with Linearize Project.')

Select a workspace with Linearize Project.


In [146]:
def select_cf(workspace):
    global custom_function
    custom_functions=workspace.get_custom_functions()
    custom_function=widgets.ToggleButtons(
    options=[custom_function.name for custom_function in custom_functions],
    value='dynamic',
    description='Custom Functions',
    disabled=False,
    )
    print('The following custom functions are available for the choosen workspace:')
    display(custom_function)
    custom_function.observe(list_cf_params,names=['value'])

def setup_model_for_execution(b):
    with output:
        global experiment_definition, custom_function, parameters
        model_name = "LinearizeExample.HeatingSystemNonLinear"
        print(f'Press execute to simulate model - {model_name} with the selected custom function.')
        model = workspace.get_model(model_name)
        # Execute experiment
        set_parameters = {i:eval(j.value) for i, j in parameters.items()}
        experiment_definition = model.new_experiment_definition(
           custom_function.with_parameters(**set_parameters)
        )
        #print(experiment_definition.to_dict(), custom_function.with_parameters(**parameters))
        return experiment_definition

def on_button_clicked(b):
    with output:
        print("Button clicked.")
        exp = workspace.execute(experiment_definition).wait()
        case = exp.get_case('case_1')
        log = case.get_log()
        log.show()
        print("Button clicked.")

def execute():
    button = widgets.Button(
        value=False,
        description='Execute',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Execute',
        icon='check' # (FontAwesome names without the `fa-` prefix)
    )
    display(button)
    button.on_click(on_button_clicked)
    
def list_cf_params(obj):
    with output:
        custom_function_name = obj['owner'].value
        global custom_function, parameters
        custom_function=workspace.get_custom_function(custom_function_name)
        
        print(f'Chosen custom function has the following parameter values. Please set your parameter choices:-')
        parameters={}
        for i,j in custom_function.parameter_values.items():
            parameters[i]=widgets.Text(
            value=str(j),
            placeholder='Type something',
            description=i,
            disabled=False   
        )
            display(parameters[i])
            parameters[i].observe(setup_model_for_execution,names=['value'])
        execute()


def on_drop_down_change(obj):
    with output:
        global workspace
        workspace = client.get_workspace(workspace_map[obj['owner'].value])
        project_names = [project.definition.name for project in workspace.get_projects()]
        #assert 'Linearize' in project_names, f'Linearize is not a workspace dependency for {workspace.definition.name}. Please choose one which has Linearize project as a dependency.'

        print("The chosen workspaces has the following projects and dependencies") 
        workspace_projects = workspace.get_projects()
        projects=widgets.Select(
            options=[project.definition.name for project in workspace_projects],
            value="Linearize",
            description='Projects',
            disabled=False,
        )


        workspace_dependencies = workspace.get_dependencies()
        dependencies=widgets.Select(
        options=[project.definition.name +' '+  project.definition.version for project in workspace_dependencies],
        value="Modelica 4.0.0",
        description='Dependencies',
        disabled=False)
        
        display(projects)
        display(dependencies)
        select_cf(workspace)
        
        
        
display(workspace_sel, output)
workspace_sel.observe(on_drop_down_change,names=['value'])    

Dropdown(description='Workspaces', options=('TestCF', 'LinearizeDemo', 'Something'), value='TestCF')

Output()